In [21]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

## Aggregate data from multiple sources

### Johns Hopkins COVID-19 data

In [22]:
# data cleaning and summing total events by country
def preprocess_summation(df, description):
    df = df.drop(['Province/State', 'Lat', 'Long'], axis=1).groupby(['Country/Region']).sum().reset_index()
    col_name = 'Total' + ' ' + description
    df[col_name] = df.apply(lambda r: r[-1], axis=1)
    df = df[['Country/Region', col_name]]
    df.columns = ['Country', col_name]
    df.replace({'Korea, South': 'South Korea', 'Czechia': 'Czech Republic', 'Taiwan*': 'Taiwan', 'US': 'United States'}, inplace=True)
    return df

In [23]:
df_confirmed = pd.read_csv('./csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
total_infections = preprocess_summation(df_confirmed, 'Infected')
total_infections.head()

,Country,Total Infected
0,Afghanistan,237
1,Albania,259
2,Algeria,847
3,Andorra,390
4,Angola,8


### Import 2018 GDP data
https://data.worldbank.org/indicator/NY.GDP.MKTP.CD

In [24]:
df_gdp = pd.read_csv('./outside_data/GDP.csv')
df_gdp = df_gdp[['Country Name', '2018']]
df_gdp.columns = ['Country', 'GDP 2018']
df_gdp.replace({'Korea, Rep.': 'South Korea'}, inplace=True)
df_gdp.shape

(264, 2)

In [25]:
df = total_infections.merge(df_gdp, how='inner', on='Country')

### Crime and Population data
https://worldpopulationreview.com/countries/crime-rate-by-country/

In [26]:
df_crime = pd.read_csv('./outside_data/crime_pop.csv', header=0, names=['Country', 'Crime Index', 'Population 2020'])
df = df.merge(df_crime, how='inner', on='Country')

### Smoking data
https://ourworldindata.org/smoking#prevalence-of-smoking-across-the-world  
Percentage of country aged 15+ who smoke

In [27]:
df_smoke = pd.read_csv('./outside_data/smoking.csv', header=0, names=['Country', 'Code', 'Year', 'Smoking 2016'])
df_smoke = df_smoke[df_smoke.apply(lambda r: r['Year'] == 2016, axis=1)]
df_smoke.drop(['Code', 'Year'], axis=1, inplace=True)
df_smoke.shape

(186, 2)

In [28]:
df = df.merge(df_smoke, how='inner', on='Country')

### Gender data
https://data.worldbank.org/indicator/SP.POP.TOTL.FE.ZS  
Percentage of females in 2018

In [29]:
df_gender = pd.read_csv('./outside_data/gender.csv')
df_gender = df_gender[['Country Name', '2018']]
df_gender.columns = ['Country', 'Females 2018']
df_gender.replace({'Korea, Rep.': 'South Korea'}, inplace=True)

In [30]:
df = df.merge(df_gender, how='inner', on='Country')

### Age data
https://worldpopulationreview.com/countries/median-age/  
Median age

In [31]:
df_age = pd.read_csv('./outside_data/age.csv')
df_age = df_age[['Place', 'Median']]
df_age.columns = ['Country', 'Median Age']
df_age.shape

(214, 2)

In [32]:
df = df.merge(df_age, how='inner', on='Country')
df.head()

,Country,Total Infected,GDP 2018,Crime Index,Population 2020,Smoking 2016,Females 2018,Median Age
0,Albania,259,1.510250e+10,40.02,2877.797,28.7,49.063095,32.9
1,Algeria,847,1.737580e+11,54.41,43851.044,15.6,49.484268,28.1
2,Argentina,1054,5.198715e+11,62.96,45195.774,21.8,51.237348,31.7
3,Armenia,571,1.243309e+10,20.78,2963.243,24.1,52.956577,35.1
4,Australia,4862,1.433904e+12,42.70,25499.884,14.7,50.199623,38.7


In [33]:
# df.to_csv('covid19.csv', index=False)

## Merge with Kaggle

In [34]:
# df_kag = pd.read_csv('./kaggle/covid19countryinfo.csv')
# df_kag
# df_kag.columns = ['Region', 'Country', *df_kag.columns[2:]]
# df_kag
# df_kag = df_kag[['Country', 'density', 'urbanpop', 'quarantine', 'schools', 'publicplace', 'gatheringlimit', 'nonessential', 'smokers', 'hospibed', 'sex0', 'sex14', 'sex25', 'sex54', 'sex64', 'sex65plus', 'sexratio', 'lung', 'femalelung', 'malelung', 'healthexp', 'firstcase']]
# df_kag.replace({'Korea, South': 'South Korea', 'Czechia': 'Czech Republic', 'Taiwan*': 'Taiwan', 'US': 'United States'}, inplace=True)

In [35]:
# df = df_kag.merge(df, how="inner", on="Country")
# df.head()

In [36]:
df_deaths = pd.read_csv('./csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
total_deaths = preprocess_summation(df_deaths, 'Deaths')
df = df.merge(total_deaths, how='inner', on='Country')

In [37]:
df_recovered = pd.read_csv('./csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
total_recover = preprocess_summation(df_recovered, 'Recovered')
df = df.merge(total_recover, how='inner', on='Country')
df.head()

,Country,Total Infected,GDP 2018,Crime Index,Population 2020,Smoking 2016,Females 2018,Median Age,Total Deaths,Total Recovered
0,Albania,259,1.510250e+10,40.02,2877.797,28.7,49.063095,32.9,15,67
1,Algeria,847,1.737580e+11,54.41,43851.044,15.6,49.484268,28.1,58,61
2,Argentina,1054,5.198715e+11,62.96,45195.774,21.8,51.237348,31.7,28,248
3,Armenia,571,1.243309e+10,20.78,2963.243,24.1,52.956577,35.1,4,31
4,Australia,4862,1.433904e+12,42.70,25499.884,14.7,50.199623,38.7,20,422


In [38]:
# df.columns = ['Country', 'Density', 'Urban Pop', 'Quarantine', 'Schools',
#        'Public Place', 'Gathering Limit', 'Nonessential', 'Smokers', 'Hospital Beds',
#        'sex0', 'sex14', 'sex25', 'sex54', 'sex64', 'sex65plus', 'Sex Ratio',
#        'Lung', 'Female Lung', 'Male Lung', 'Health Exp', 'First Case',
#        'Total Infected', 'GDP 2018', 'Crime Index', 'Population 2020',
#        'Smoking 2016', 'Females 2018', 'Median Age', 'Total Deaths',
#        'Total Recovered']

df.head()

,Country,Total Infected,GDP 2018,Crime Index,Population 2020,Smoking 2016,Females 2018,Median Age,Total Deaths,Total Recovered
0,Albania,259,1.510250e+10,40.02,2877.797,28.7,49.063095,32.9,15,67
1,Algeria,847,1.737580e+11,54.41,43851.044,15.6,49.484268,28.1,58,61
2,Argentina,1054,5.198715e+11,62.96,45195.774,21.8,51.237348,31.7,28,248
3,Armenia,571,1.243309e+10,20.78,2963.243,24.1,52.956577,35.1,4,31
4,Australia,4862,1.433904e+12,42.70,25499.884,14.7,50.199623,38.7,20,422


In [39]:
cols = df.columns.tolist()
total_infected_index = list(df.columns).index('Total Infected')
cols[len(cols)-3], cols[total_infected_index] = cols[total_infected_index], cols[len(cols)-3]
df = df[cols]
df.head()
df[df.Country == 'India']

,Country,Median Age,GDP 2018,Crime Index,Population 2020,Smoking 2016,Females 2018,Total Infected,Total Deaths,Total Recovered
41,India,27.9,2.718732e+12,42.38,1380004.385,11.5,48.02354,1998,58,148


In [40]:
# df.to_csv('covid19_by_country.csv', index=False)